In [1]:
from scholarly import scholarly
import jsonpickle
import json
from datetime import datetime
import os


In [2]:
author: dict = scholarly.search_author_id('4DbuyrgAAAAJ')
scholarly.fill(author, sections=['basics', 'indices', 'counts', 'publications'])
name = author['name']
author['updated'] = str(datetime.now())
author['publications'] = {v['author_pub_id']:v for v in author['publications']}
print(json.dumps(author, indent=2))
os.makedirs('results', exist_ok=True)
with open(f'results/gs_data.json', 'w') as outfile:
    json.dump(author, outfile, ensure_ascii=False)


{
  "container_type": "Author",
  "filled": [
    "basics",
    "publications",
    "indices",
    "counts"
  ],
  "scholar_id": "4DbuyrgAAAAJ",
  "source": "AUTHOR_PROFILE_PAGE",
  "name": "Daniel Rodriguez-Cardenas",
  "affiliation": "William & Mary",
  "organization": 12917003543313146137,
  "interests": [
    "Computer Science"
  ],
  "email_domain": "@wm.edu",
  "homepage": "https://danielrcardenas.github.io/",
  "citedby": 65,
  "publications": {
    "4DbuyrgAAAAJ:8k81kl-MbHgC": {
      "container_type": "Publication",
      "source": "AUTHOR_PUBLICATION_ENTRY",
      "bib": {
        "title": "Benchmarking causal study to interpret large language models for source code",
        "pub_year": "2023"
      },
      "filled": false,
      "author_pub_id": "4DbuyrgAAAAJ:8k81kl-MbHgC",
      "num_citations": 21,
      "citedby_url": "https://scholar.google.com/scholar?oi=bibs&hl=en&cites=14523889433872554206",
      "cites_id": [
        "14523889433872554206"
      ]
    },
    "4Dbu

In [3]:
shieldio_data = {
  "schemaVersion": 1,
  "label": "citations",
  "message": f"{author['citedby']}",
}
with open(f'results/gs_data_shieldsio.json', 'w') as outfile:
    json.dump(shieldio_data, outfile, ensure_ascii=False)


In [4]:
shieldio_data_mtl = {
  "schemaVersion": 1,
  "label": "citations",
  "message": f"{author['publications']['Uf9GqRsAAAAJ:bEWYMUwI8FkC']['num_citations']}",
}
with open(f'results/gs_data_shieldsio_mtl.json', 'w') as outfile:
    json.dump(shieldio_data_mtl, outfile, ensure_ascii=False)
    

KeyError: 'Uf9GqRsAAAAJ:bEWYMUwI8FkC'

In [5]:
shieldio_data_mnemonics = {
  "schemaVersion": 1,
  "label": "citations",
  "message": f"{author['publications']['Uf9GqRsAAAAJ:k_IJM867U9cC']['num_citations']}",
}
with open(f'results/gs_data_shieldsio_mnemonics.json', 'w') as outfile:
    json.dump(shieldio_data_mnemonics, outfile, ensure_ascii=False)

KeyError: 'Uf9GqRsAAAAJ:k_IJM867U9cC'

In [ ]:
shieldio_data_aanets = {
  "schemaVersion": 1,
  "label": "citations",
  "message": f"{author['publications']['Uf9GqRsAAAAJ:u_35RYKgDlwC']['num_citations']}",
}
with open(f'results/gs_data_shieldsio_aanets.json', 'w') as outfile:
    json.dump(shieldio_data_aanets, outfile, ensure_ascii=False)
    

In [ ]:
shieldio_data_e3bm = {
  "schemaVersion": 1,
  "label": "citations",
  "message": f"{author['publications']['Uf9GqRsAAAAJ:vV6vV6tmYwMC']['num_citations']}",
}
with open(f'results/gs_data_shieldsio_e3bm.json', 'w') as outfile:
    json.dump(shieldio_data_e3bm, outfile, ensure_ascii=False)
   

In [ ]:
shieldio_data_lst = {
  "schemaVersion": 1,
  "label": "citations",
  "message": f"{author['publications']['Uf9GqRsAAAAJ:TFP_iSt0sucC']['num_citations']}",
}
with open(f'results/gs_data_shieldsio_lst.json', 'w') as outfile:
    json.dump(shieldio_data_lst, outfile, ensure_ascii=False)
